# 1) IMPORTS

In [95]:
!pip install findspark

In [96]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.5.3-bin-hadoop3'

In [97]:
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [108]:
spark = SparkSession \
    .builder \
    .appName("Protob Conversion to Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [99]:
import requests
import json
import math
import datetime
from pyspark.sql import functions as F
from pyspark.sql import types as T

# 2) Variables

In [100]:
load_date = datetime.datetime.now().date()

# 3) REQUEST

## 3.1) Finding out the number of breweries

In [101]:
url = 'https://api.openbrewerydb.org/v1/breweries/meta'

response = requests.request("GET", url = url)

num_brew = int(response.json()['total'])

## 3.2) Getting data

In [102]:
num_req = math.ceil((num_brew/200))

list_brew = []

for i in range(0, num_req + 1):
    url = f'https://api.openbrewerydb.org/v1/breweries?page={i}&per_page=200'

    response = requests.request("GET", url = url)

    list_brew.extend(response.json())

## 3.3) Writing data in Bronze layer

In [103]:
with open("/Users/nelso/Desktop/CaseDE/datalake/bronze/raw.json", "w") as outfile:
    json.dump(list_brew, outfile)

# 4) Transforming data

## 4.1) Silver layer

In [104]:
df_bl = spark.read.json("/Users/nelso/Desktop/CaseDE/datalake/bronze/raw.json")

In [105]:
df_bl = df_bl.drop_duplicates()

In [106]:
df_bl = df_bl\
    .select\
        (
            F.col("id"),
            F.col("address_1"),
            F.col("address_2"),
            F.col("address_3"),
            F.col("brewery_type"),
            F.col("city"),
            F.col("country"),
            F.col("latitude"),
            F.col("longitude"),
            F.col("name"),
            F.col("phone"),
            F.col("postal_code"),
            F.col("state"),
            F.col("state_province"),
            F.col("street"),
            F.col("website_url"),
            F.lit(True).alias("active"),
            F.lit(load_date).alias("load_date")
        )

Since the column [id] is a PK, we can use it to check if a brewery inactivated

In [52]:
# Loading the data in the silver layer
try:
    df_sl = spark.read.format("parquet").load("/Users/nelso/Desktop/CaseDE/datalake/silver/HistoryBreweries.parquet")
except:
    df_sl = df_bl.filter("1 = 0")

In [111]:
df_bl

DataFrame[id: string, address_1: string, address_2: string, address_3: string, brewery_type: string, city: string, country: string, latitude: string, longitude: string, name: string, phone: string, postal_code: string, state: string, state_province: string, street: string, website_url: string, active: boolean, load_date: date]

In [117]:
df_bl.write.mode("overwrite").parquet(r'C:\Users\nelso\Desktop\CaseDE\datalake\silver\historybreweries.parquet')

Py4JJavaError: An error occurred while calling o738.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fi

In [60]:
df_inactive = df_bl.alias("a")\
    .join\
        (
            df_sl.alias("b"),
            df_sl.id == df_bl.id,
            how="left"
        )\
    .select\
        (
            df_sl.id,
            df_sl.address_1,
            df_sl.address_2,
            df_sl.address_3,
            df_sl.brewery_type,
            df_sl.city,
            df_sl.country,
            df_sl.latitude,
            df_sl.longitude,
            df_sl.name,
            df_sl.phone,
            df_sl.postal_code,
            df_sl.state,
            df_sl.state_province,
            df_sl.street,
            df_sl.website_url,
            F.lit(False).alias("active"),
            df_sl.load_date
        )\
    .filter(df_bl.id.isNull())

AnalysisException: Column phone#466, name#465, postal_code#467, brewery_type#459, address_3#458, address_1#456, address_2#457, latitude#463, longitude#464, id#462, city#460, website_url#471, street#470, state_province#469, country#461, load_date#490, state#468 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.